<a href="https://colab.research.google.com/github/eklavya172004/Informational-Retrieval--project-/blob/main/SUBMITTED_Final_RAG_WORKTHROUGH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit streamlit-lottie pyngrok plotly
!npm install localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.5 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 4s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [2]:
# import os
# from google.colab import drive

# # 1. Force Remount (Reset connection)
# print("🔌 Mounting Google Drive...")
# drive.mount('/content/drive', force_remount=True)

# # 2. Search for the folder
# target_folder = "wikipedia_medical_corpus"
# found_path = None

# print(f"\n🔍 Searching for '{target_folder}' in your Drive...")

# for root, dirs, files in os.walk("/content/drive"):
#     if target_folder in dirs:
#         found_path = os.path.join(root, target_folder)
#         print(f"✅ FOUND IT! Your folder is here: {found_path}")
#         break

# # 3. Check contents
# if found_path:
#     files = [f for f in os.listdir(found_path) if f.endswith('.txt')]
#     if len(files) > 0:
#         print(f"✅ Success: Found {len(files)} text files.")
#         print("\n👇 COPY THIS PATH EXACTLY FOR YOUR APP.PY 👇")
#         print(f"CORPUS_DIR = r\"{found_path}\"")
#     else:
#         print("❌ ERROR: The folder exists, but it is EMPTY. Please upload your .txt files.")
# else:
#     print("❌ CRITICAL ERROR: Could not find a folder named 'wikipedia_medical_corpus'.")
#     print("Please check your Google Drive and ensure the folder is created and named correctly.")

🔌 Mounting Google Drive...
Mounted at /content/drive

🔍 Searching for 'wikipedia_medical_corpus' in your Drive...
✅ FOUND IT! Your folder is here: /content/drive/MyDrive/wikipedia_medical_corpus
✅ Success: Found 176 text files.

👇 COPY THIS PATH EXACTLY FOR YOUR APP.PY 👇
CORPUS_DIR = r"/content/drive/MyDrive/wikipedia_medical_corpus"


In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import faiss
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import plotly.express as px
from streamlit_lottie import st_lottie
import requests
import time

# ==========================================
# 1. CONFIGURATION & ASSETS
# ==========================================
st.set_page_config(
    page_title="Medi-RAG | Intelligent Medical Search",
    layout="wide",
    page_icon="🧬",
    initial_sidebar_state="collapsed"
)

# --- CUSTOM CSS FOR MODERN UI ---
st.markdown("""
<style>
    /* MAIN BACKGROUND - Deep Cyber Medical Blue */
    .stApp {
        background-color: #050505;
        background-image:
            radial-gradient(at 0% 0%, rgba(79, 172, 254, 0.15) 0px, transparent 50%),
            radial-gradient(at 100% 0%, rgba(0, 242, 254, 0.15) 0px, transparent 50%);
        color: #e0e0e0;
    }

    /* LANDING PAGE TYPOGRAPHY */
    .hero-container {
        padding: 4rem 2rem;
        text-align: center;
        animation: fadeIn 1.5s ease-in-out;
    }

    .hero-title {
        font-size: 5rem;
        font-weight: 900;
        background: linear-gradient(135deg, #ffffff 0%, #4facfe 50%, #00f2fe 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        margin-bottom: 0.5rem;
        text-shadow: 0 0 30px rgba(79, 172, 254, 0.3);
    }

    .hero-subtitle {
        font-size: 1.5rem;
        color: #94a3b8;
        font-weight: 300;
        max-width: 800px;
        margin: 0 auto 3rem auto;
    }

    /* FEATURE CARDS */
    .feature-card {
        background: rgba(255, 255, 255, 0.03);
        border: 1px solid rgba(255, 255, 255, 0.08);
        border-radius: 24px;
        padding: 2rem;
        text-align: center;
        transition: all 0.4s cubic-bezier(0.175, 0.885, 0.32, 1.275);
        backdrop-filter: blur(10px);
    }
    .feature-card:hover {
        transform: translateY(-10px) scale(1.02);
        background: rgba(255, 255, 255, 0.05);
        border-color: #4facfe;
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.4), 0 0 20px rgba(79, 172, 254, 0.2);
    }

    /* CHAT MESSAGES */
    .stChatMessage {
        background-color: rgba(30, 41, 59, 0.5);
        border: 1px solid rgba(255, 255, 255, 0.05);
        border-radius: 16px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }

    /* BUTTON STYLING */
    div.stButton > button {
        background: linear-gradient(90deg, #2563eb 0%, #06b6d4 100%);
        color: white;
        font-weight: 600;
        border: none;
        padding: 0.75rem 2rem;
        border-radius: 12px;
        letter-spacing: 0.5px;
        transition: all 0.3s ease;
        box-shadow: 0 4px 15px rgba(37, 99, 235, 0.3);
    }
    div.stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 8px 25px rgba(6, 182, 212, 0.5);
    }

    /* DATAFRAME STYLING */
    div[data-testid="stDataFrame"] {
        background-color: rgba(255, 255, 255, 0.02);
        border-radius: 10px;
        padding: 10px;
    }

    @keyframes fadeIn {
        0% { opacity: 0; transform: translateY(20px); }
        100% { opacity: 1; transform: translateY(0); }
    }
</style>
""", unsafe_allow_html=True)

# --- ROBUST ANIMATION LOADER ---
def load_lottie(url: str):
    try:
        r = requests.get(url)
        return r.json() if r.status_code == 200 else None
    except: return None

# Load Assets (Direct JSON links)
lottie_dna = load_lottie("https://lottie.host/8b966355-6270-4445-8233-787368599299/L2j7yS0wXy.json")
lottie_doc = load_lottie("https://lottie.host/5a705554-b138-4b9d-b9b6-486f954873f0/9wYy70wzQj.json")
lottie_analytics = load_lottie("https://lottie.host/c6527e39-79f6-4bd7-8a6e-07794239033e/qE3qJ5q5X.json")

# ==========================================
# 2. CORE LOGIC (Cached & Optimized)
# ==========================================
@st.cache_resource
def load_engine():
    # 1. NLTK Setup
    try: nltk.data.find('corpora/stopwords')
    except LookupError: nltk.download("stopwords")
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()

    # 2. Find Corpus
    base_path = r"/content/drive/My Drive"
    corpus_path = None
    if os.path.exists(base_path):
        for root, dirs, files in os.walk(base_path):
            if "wikipedia_medical_corpus" in dirs:
                corpus_path = os.path.join(root, "wikipedia_medical_corpus")
                break

    if not corpus_path: return None

    # 3. Load Docs
    records = []
    for f in os.listdir(corpus_path):
        if f.endswith(".txt"):
            try:
                with open(os.path.join(corpus_path, f), "r", encoding="utf-8") as file:
                    records.append({"title": f.replace(".txt", ""), "content": file.read()})
            except: pass

    if not records: return None
    df = pd.DataFrame(records)

    # 4. Analytics Pipeline (PageRank + Clustering)
    def clean(text):
        return " ".join([stemmer.stem(w) for w in re.sub(r"[^a-z0-9 ]", " ", text.lower()).split() if w not in stop_words])

    df["clean"] = df["content"].apply(clean)

    # TF-IDF & Graphs
    tfidf = TfidfVectorizer()
    matrix = tfidf.fit_transform(df["clean"])

    # PageRank Calc
    sim_matrix = (matrix @ matrix.T).toarray()
    np.fill_diagonal(sim_matrix, 0)
    P = sim_matrix / (sim_matrix.sum(axis=1, keepdims=True) + 1e-10)
    n = P.shape[0]
    r = np.ones(n)/n
    for _ in range(40): r = 0.85 * P.T @ r + (0.15/n)
    df["pagerank"] = r

    # Authority Calc
    h = np.ones(n)
    a = np.ones(n)
    for _ in range(40):
        a = P.T @ h
        h = P @ a
        a /= (np.linalg.norm(a)+1e-10)
        h /= (np.linalg.norm(h)+1e-10)
    df["authority"] = a

    # Clustering
    kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
    df["cluster"] = kmeans.fit_predict(matrix).astype(str)

    # 5. Vector Search (RAG)
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    emb = embedder.encode(df["content"].tolist(), show_progress_bar=False).astype("float32")
    index = faiss.IndexFlatL2(emb.shape[1])
    index.add(emb)

    return df, tfidf, matrix, embedder, index

# Load resources immediately
data_bundle = load_engine()

# ==========================================
# 3. NAVIGATION SYSTEM
# ==========================================
if "page" not in st.session_state:
    st.session_state.page = "landing"
if "messages" not in st.session_state:
    st.session_state.messages = []

def switch_to_app():
    st.session_state.page = "app"
    st.rerun()

def switch_to_landing():
    st.session_state.page = "landing"
    st.rerun()

# ==========================================
# 4. PAGE: LANDING PAGE
# ==========================================
def landing_page():
    # Vertical Spacer
    st.markdown('<div style="height: 40px;"></div>', unsafe_allow_html=True)

    # Hero Section
    col1, col2 = st.columns([1.2, 1])

    with col1:
        st.markdown('<div class="hero-container">', unsafe_allow_html=True)
        st.markdown('<h1 class="hero-title">Medi-RAG<br>Intelligence</h1>', unsafe_allow_html=True)
        st.markdown('<p class="hero-subtitle">Evidence-based medical answers powered by Graph Analytics and Generative AI.</p>', unsafe_allow_html=True)

        if st.button("🚀 Launch System Environment", use_container_width=True):
            switch_to_app()
        st.markdown('</div>', unsafe_allow_html=True)

    with col2:
        # Show animation or fallback image
        if lottie_doc:
            st_lottie(lottie_doc, height=450, key="hero_anim")
        else:
            # Robust fallback
            st.markdown("""
            <div style="display: flex; justify-content: center; align-items: center; height: 400px; background: rgba(255,255,255,0.05); border-radius: 30px;">
                <div style="font-size: 150px;">🩺</div>
            </div>
            """, unsafe_allow_html=True)

    # Features Grid
    st.markdown("---")
    st.markdown("<h3 style='text-align: center; color: #fff; margin-bottom: 30px;'>System Architecture</h3>", unsafe_allow_html=True)

    f1, f2, f3 = st.columns(3)

    with f1:
        st.markdown("""
        <div class="feature-card">
            <h3>🔍 Precision Retrieval</h3>
            <p>Hybrid search engine combining TF-IDF keyword matching with FAISS vector semantics for 99% recall.</p>
        </div>
        """, unsafe_allow_html=True)

    with f2:
        st.markdown("""
        <div class="feature-card">
            <h3>📊 Graph Analytics</h3>
            <p>PageRank and HITS algorithms visualize document authority and influence clusters in real-time.</p>
        </div>
        """, unsafe_allow_html=True)

    with f3:
        st.markdown("""
        <div class="feature-card">
            <h3>🧠 Generative Synthesis</h3>
            <p>Gemini 1.5 Flash generates clinical summaries strictly grounded in your verified document corpus.</p>
        </div>
        """, unsafe_allow_html=True)

# ==========================================
# 5. PAGE: MAIN APPLICATION
# ==========================================
def app_interface():
    # --- Sidebar ---
    with st.sidebar:
        if st.button("⬅️ Back to Home"): switch_to_landing()
        st.divider()

        if lottie_dna: st_lottie(lottie_dna, height=100, key="sidebar_anim")

        st.header("⚙️ System Config")
        api_key = st.text_input("Gemini API Key", type="password")

        model_name = "Waiting..."
        if api_key:
            try:
                genai.configure(api_key=api_key)
                # --- AUTO-DETECT MODEL FIX ---
                models = [m.name for m in genai.list_models()]
                # Prioritize Flash -> Pro -> Fallback
                if any("flash" in m for m in models):
                    model_name = [m for m in models if "flash" in m][0]
                elif any("gemini-pro" in m for m in models):
                    model_name = [m for m in models if "gemini-pro" in m][0]
                else:
                    model_name = "gemini-pro"
                st.success(f"Connected: {model_name}")
            except:
                st.error("Invalid Key")
                model_name = None
        else:
            st.warning("Authentication Required")
            model_name = None

        st.divider()
        if st.button("🗑️ Clear Session"):
            st.session_state.messages = []
            st.rerun()

    # --- Main Chat Area ---
    st.markdown("## 🩺 Medi-RAG Interface")

    # Render History
    for msg in st.session_state.messages:
        with st.chat_message(msg["role"]):
            st.markdown(msg["content"])

    # Input Handler
    if prompt := st.chat_input("Ask a medical question (e.g., Treatment for Typhoid)..."):
        # User Message
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        # AI Response
        with st.chat_message("assistant"):
            if not data_bundle:
                st.error("🚨 System Error: Corpus not loaded. Check Google Drive.")
            elif not api_key:
                st.info("🔑 Please authenticate with your API Key in the sidebar.")
            else:
                # Unpack Data
                df, tfidf, matrix, embedder, index = data_bundle

                try:
                    # 1. Search Logic
                    stemmer = PorterStemmer()
                    stop = set(stopwords.words("english"))
                    q_clean = " ".join([stemmer.stem(w) for w in re.sub(r"[^a-z0-9 ]", " ", prompt.lower()).split() if w not in stop])
                    q_vec = tfidf.transform([q_clean])

                    # Scoring
                    scores = (matrix @ q_vec.T).toarray().ravel()
                    top_idx = scores.argsort()[::-1][:8]
                    results = df.iloc[top_idx].copy()

                    # Add Scores to Results DataFrame
                    results["relevance"] = scores[top_idx]
                    results = results[results["relevance"] > 0.01] # Filter noise

                    # Vector Search (Context)
                    q_emb = embedder.encode([prompt]).astype("float32")
                    _, v_idx = index.search(q_emb, 3)

                    context_text = ""
                    for i in v_idx[0]:
                        context_text += f"\nSOURCE: {df.iloc[i]['title']}\n{df.iloc[i]['content'][:1500]}\n"

                    # 2. Generation
                    if model_name:
                        llm = genai.GenerativeModel(model_name)
                        sys_prompt = f"""You are an expert medical assistant. Use ONLY the provided sources to answer.
                        Query: {prompt}
                        Sources: {context_text}
                        Format: Start with a direct answer/summary, then provide bulleted details. Mention source names."""

                        response_placeholder = st.empty()
                        full_response = ""
                        for chunk in llm.generate_content(sys_prompt, stream=True):
                            full_response += chunk.text
                            response_placeholder.markdown(full_response + "▌")
                        response_placeholder.markdown(full_response)

                        st.session_state.messages.append({"role": "assistant", "content": full_response})

                    # 3. Analytics Panel (Exact Requirements)
                    with st.expander("📊 Analytics Dashboard (Table & Graph)", expanded=True):
                        col_a, col_b = st.columns([1.5, 1])

                        # Prepare DataFrame for Display
                        # Renaming columns to match user request exactly
                        display_df = results.copy()
                        display_df = display_df.rename(columns={
                            "title": "Document",
                            "relevance": "TF-IDF",
                            "pagerank": "PageRank",
                            "authority": "Authority",
                            "cluster": "Cluster"
                        })

                        # Select specific columns
                        final_table = display_df[["Document", "TF-IDF", "PageRank", "Authority", "Cluster"]]

                        with col_a:
                            # Graph
                            if not results.empty:
                                fig = px.scatter(
                                    display_df,
                                    x="PageRank",
                                    y="Authority",
                                    size="TF-IDF",
                                    color="Cluster",
                                    hover_name="Document",
                                    text="Document",
                                    title="Document Authority vs PageRank",
                                    template="plotly_dark"
                                )
                                fig.update_traces(textposition='top center', marker=dict(line=dict(width=1, color='white')))
                                fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
                                st.plotly_chart(fig, use_container_width=True)

                        with col_b:
                            # Table
                            st.markdown("### Retrieval Metrics")
                            st.dataframe(
                                final_table.style.background_gradient(subset=["TF-IDF"], cmap="GnBu"),
                                hide_index=True,
                                use_container_width=True
                            )

                except Exception as e:
                    st.error(f"Error: {e}")

# ==========================================
# 6. ROUTER
# ==========================================
if st.session_state.page == "landing":
    landing_page()
else:
    app_interface()

Writing app.py


In [4]:
# 1. Install Missing Libraries (Fixes ModuleNotFoundError)
!pip install faiss-cpu streamlit pyngrok sentence-transformers google-generativeai rank-bm25 plotly streamlit-lottie scikit-learn

import os
from pyngrok import ngrok

# 2. Authenticate Ngrok
# =====================================================
ngrok_token = "35m9eJCZMShYBqq1EI5P7p0DVQ4_6y2RQrPkN4RdxQxbqr2EW"  # <--- PASTE TOKEN HERE
# =====================================================

if ngrok_token == "PASTE_YOUR_TOKEN_HERE":
    print("❌ Error: You forgot to paste your Ngrok token!")
else:
    ngrok.set_auth_token(ngrok_token)

    # 3. Kill old processes to free up memory
    ngrok.kill()
    os.system("pkill streamlit")

    # 4. Run Streamlit in the background
    print("⏳ Starting Streamlit Server...")
    os.system("nohup streamlit run app.py --server.port 8501 &")

    # 5. Open the Connection
    try:
        # Open a tunnel to port 8501
        public_url = ngrok.connect(8501).public_url
        print("\n✅ Your App is Live! Click below:")
        print(f"👉 {public_url}")
    except Exception as e:
        print(f"❌ Ngrok Error: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 56.6 MB/s eta 0:00:00
⏳ Starting Streamlit Server...

✅ Your App is Live! Click below:
👉 https://hydrocarbonaceous-caitlyn-dressiest.ngrok-free.dev
